In [1]:
import os
file_path = "Intermezzo.txt"
with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()
print(text[:500])

	Лишилось тільки ще спакуватись... Се було одно з тих незчисленних «треба», які мене так утомили і не давали спати. Дарма, чи те «треба» мале, чи велике, — вагу те має, що кожен раз воно вимагає уваги, що не я їм, а воно мною уже керує. Фактично стаєш невільником сього многоголового звіра. Хоч на час увільнитись від нього, забути, спочити. Я утомився. 
	Бо життя безупинно і невблаганно іде на мене, як хвиля на берег. Не тільки власне, а і чуже. А врешті — хіба я знаю, де кінчається власне життя,


In [2]:
import re
from tensorflow.keras.preprocessing.text import Tokenizer

# Очищення тексту/видалення зайвих символів
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^а-яіїєґ' ]", "", text)  
    return text
    
cleaned_text = clean_text(text)
print(cleaned_text)

# Токенізація тексту
tokenizer = Tokenizer(char_level=False, oov_token="<OOV>")
tokenizer.fit_on_texts([cleaned_text])

sequences = tokenizer.texts_to_sequences([cleaned_text])

word_index = tokenizer.word_index
print(len(word_index))
print(sequences[0][:10])

лишилось тільки ще спакуватись се було одно з тих незчисленних треба які мене так утомили і не давали спати дарма чи те треба мале чи велике  вагу те має що кожен раз воно вимагає уваги що не я їм а воно мною уже керує фактично стаєш невільником сього многоголового звіра хоч на час увільнитись від нього забути спочити я утомився бо життя безупинно і невблаганно іде на мене як хвиля на берег не тільки власне а і чуже а врешті  хіба я знаю де кінчається власне життя а чуже починається я чую як чуже існування входить в моє мов повітря крізь вікна і двері як води притоків у річку я не можу розминутись з людиною я не можу бути самотнім признаюсь  заздрю планетам вони мають свої орбіти і ніщо не стає їм на їхній дорозі тоді як на своїй я скрізь і завжди стрічаю людину так ти стаєш мені на дорозі і уважаєш що маєш на мене право ти скрізь се ти одягла землю в камінь й залізо се ти через вікна будинків  тисячі чорних ротів  вічно дихаєш смородом ти бичуєш святу тишу землі скреготом фабрик громо

In [4]:
import numpy as np

# Створення послідовностей для навчання
def create_sequences(seq, seq_length):
    inputs = []
    targets = []
    for i in range(0, len(seq[0]) - seq_length):
        inputs.append(seq[0][i:i+seq_length])
        targets.append(seq[0][i+seq_length])
    return np.array(inputs), np.array(targets)

seq_lenghth = 10  # Довжина вхідної послідовності
X, y = create_sequences(sequences, seq_lenghth)

print(X.shape)
print(y.shape)

(3872, 10)
(3872,)


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

vocab_size = len(word_index) + 1 
embedding_dim = 128

# Побудова моделі
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=seq_lenghth),
    LSTM(256, return_sequences=False),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Навчання моделі
model.fit(X, y, epochs=40, batch_size=64)

Epoch 1/40
61/61 [==============================] - 4s 29ms/step - loss: 7.3079 - accuracy: 0.0325
Epoch 2/40
61/61 [==============================] - 2s 29ms/step - loss: 6.8571 - accuracy: 0.0367
Epoch 3/40
61/61 [==============================] - 2s 29ms/step - loss: 6.7220 - accuracy: 0.0367
Epoch 4/40
61/61 [==============================] - 2s 28ms/step - loss: 6.6115 - accuracy: 0.0367
Epoch 5/40
61/61 [==============================] - 2s 29ms/step - loss: 6.4809 - accuracy: 0.0367
Epoch 6/40
61/61 [==============================] - 2s 29ms/step - loss: 6.3640 - accuracy: 0.0369
Epoch 7/40
61/61 [==============================] - 2s 28ms/step - loss: 6.2487 - accuracy: 0.0372
Epoch 8/40
61/61 [==============================] - 2s 28ms/step - loss: 6.1257 - accuracy: 0.0390
Epoch 9/40
61/61 [==============================] - 2s 28ms/step - loss: 5.9950 - accuracy: 0.0436
Epoch 10/40
61/61 [==============================] - 2s 29ms/step - loss: 5.8323 - accuracy: 0.0467
Epoch 11/

In [18]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences

# Функція для температурного семплінгу
def sample_with_temperature(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions + 1e-10) / temperature  
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds) 
    return np.random.choice(len(predictions), p=predictions)

# Функція для top-k sampling
def top_k_sampling(predictions, k=5):
    top_k_indices = np.argsort(predictions)[-k:]  
    top_k_probs = predictions[top_k_indices]
    top_k_probs /= np.sum(top_k_probs)  
    return np.random.choice(top_k_indices, p=top_k_probs)

# Функція для генерації тексту
def generate_text(model, tokenizer, seed_text, length, seq_length, temperature=1.0, k=None):
    for _ in range(length):
        # Токенізуємо seed текст
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=seq_length, padding='pre')
        # Прогнозуємо ймовірності наступного слова
        predictions = model.predict(token_list, verbose=0)[0]
        # Вибираємо метод семплінгу
        if k:
            predicted_index = top_k_sampling(predictions, k=k)  
        else:
            predicted_index = sample_with_temperature(predictions, temperature=temperature)  
        # Отримуємо слово з індексу
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                seed_text += " " + word
                break

    return seed_text

print(generate_text(model, tokenizer, "серед поля", 50, seq_lenghth, temperature=0.7, k=5))

серед поля на себе свою душу і засівають вона м'яких зозуля то мої руки спокійно ще навіть в серце а він мені злість в ведмеді ліниво він у тобі раз життя хто щось маю так мені бруд наче сім спиною теж їм а вони шріт пава я легким спадають горять він світить
